In [1]:
import obsidian
print(f'obsidian version: ' + obsidian.__version__)

import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

obsidian version: 0.8.4


In [2]:
from obsidian import Campaign, Target, ParamSpace, BayesianOptimizer
from obsidian.parameters import Param_Continuous, Param_Categorical

In [3]:
params = [
    Param_Continuous('Volume', 10, 30),
    Param_Continuous('Metal', 0.01, 0.05),
    Param_Continuous('ML_Ratio', 0.1, 0.5),
    Param_Continuous('Reagent', 1, 3),
    Param_Continuous('Other1', 0, 1),
    Param_Continuous('Other2', 0, 1),
    Param_Categorical('Ligand', ['A', 'B']),
    ]

X_space = ParamSpace(params)
target = [Target('AY', aim='max'),
          Target('ee', aim='max')]
campaign = Campaign(X_space, target, seed=0)
X0 = campaign.initialize(m_initial = 10, method = 'LHS')

X0

,Volume,Metal,ML_Ratio,Reagent,Other1,Other2,Ligand
0,29.0,0.012,0.32,1.9,0.25,0.75,A
1,21.0,0.048,0.24,2.1,0.05,0.45,A
2,23.0,0.032,0.48,2.9,0.35,0.65,B
3,27.0,0.028,0.20,2.3,0.95,0.35,B
4,13.0,0.024,0.40,1.7,0.15,0.05,B
5,15.0,0.044,0.12,1.5,0.55,0.55,B
6,19.0,0.020,0.28,1.1,0.85,0.95,B
7,11.0,0.040,0.36,2.5,0.75,0.85,A
8,25.0,0.036,0.44,1.3,0.65,0.25,A
9,17.0,0.016,0.16,2.7,0.45,0.15,A


In [4]:
from obsidian.experiment import Simulator
from obsidian.experiment.benchmark import shifted_parab
import numpy as np

simulator = Simulator(X_space, shifted_parab, name='AY', eps=0.05)
y0 = simulator.simulate(X0)


Z0 = pd.concat([X0, y0], axis=1)
Z0['ee'] = Z0['AY'] / 100 + np.random.normal(0, 0.05, len(Z0))
Z0

campaign.add_data(Z0)
campaign.data.sample(5)

,Volume,Metal,ML_Ratio,Reagent,Other1,Other2,Ligand,AY,ee,Iteration
Observation ID,,,,,,,,,,
8,25.0,0.036,0.44,1.3,0.65,0.25,A,50.834096,0.523994,0
1,21.0,0.048,0.24,2.1,0.05,0.45,A,59.102087,0.663735,0
5,15.0,0.044,0.12,1.5,0.55,0.55,B,60.733816,0.624022,0
6,19.0,0.020,0.28,1.1,0.85,0.95,B,52.017530,0.594879,0
3,27.0,0.028,0.20,2.3,0.95,0.35,B,50.976796,0.515852,0


In [5]:
campaign.fit()

GP model has been fit                       to data with a train-score of: 1 for response: AY
GP model has been fit                       to data with a train-score of: 1 for response: ee


In [6]:
from cogs_obj import COGS_Objective

In [11]:
cogs_obj = COGS_Objective(X_space, target)

campaign.set_objective(cogs_obj)

IncompatibleObjectiveError: Objective(s) did not successfully execute on sample

In [9]:
campaign.o

IncompatibleObjectiveError: Objective(s) did not successfully execute on sample